In [34]:
pip install requests python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [61]:
STACK_EXCHANGE_API_KEY = 'rl_2AK3hrozTLChbcAFfTMnbu2km'

In [39]:
# import os
# import requests
# from datetime import datetime, timedelta
# import time
# from bs4 import BeautifulSoup as bs

# def fetch_stackoverflow_threads(tags, question, view_count=1, days=365*5):
#     API_KEY = STACK_EXCHANGE_API_KEY
#     base_url = "https://api.stackexchange.com/2.3/search/advanced"
    
#     from_date = datetime.now() - timedelta(days=days)
#     timestamp = time.mktime(from_date.timetuple())
    
#     params = {
#         "q": question,
#         "order": "desc",
#         "sort": "votes",
#         "tagged": ";".join(tags),
#         "site": "stackoverflow",
# #         "fromdate": int(timestamp),
#         "key": API_KEY,
#         "filter": "!0WRXThY5-I_TB(WxIdhBDpR5e"
#     }
    
    
#     try:
#         response = requests.get(base_url, params=params)
#         print(response.url)
#         response.raise_for_status()
#         data = response.json()
        
#         threads = []
#         for item in data.get("items", []):
#             if item["view_count"] >= view_count:
#                 # Extract question and top answer
#                 thread = {
#                     "title": item["title"],
#                     "question_body": item["body"],
#                     "link": item["link"],
#                     "score": item["score"],
#                     "tags": item["tags"],
#                     "question_id": item["question_id"],
#                     "answer_count": item["answer_count"],
#                     "view_count": item["view_count"],
#                     "answers": []
#                 }

#                 if item["is_answered"]:
#                     answer_url = f'https://api.stackexchange.com/2.3/questions/{item["question_id"]}/answers'
#                     params = {
#                         'order': 'asc',
#                         'sort': 'votes',    
#                         'site': 'stackoverflow',       
#                         'filter': '!*Mg4Pjg8kf1X_aIL',  
#                     }
#                     response = requests.get(answer_url, params=params)
#                     answers = response.json()
                    
#                     cleaned_answers = ''
#                     for answer in answers['items']:
#                         if answer['score'] > 0:
#                             soup = bs(answer['body'], 'html.parser')
#                             clean_text = soup.get_text()
#                             thread["answers"].append({
#                                 "score": answer["score"],
#                                 "body": clean_text.strip()
#                             })
                
#                 threads.append(thread)
        
#         return threads
    
#     except requests.exceptions.RequestException as e:
#         print(f"API request failed: {e}")
#         return []

In [79]:
import os
import requests
from datetime import datetime, timedelta
import time
from bs4 import BeautifulSoup as bs

def fetch_stackoverflow_threads(tags, question, min_answer_count=1):
    page = 1
    has_more = True
    API_KEY = STACK_EXCHANGE_API_KEY
    base_url = "https://api.stackexchange.com/2.3/search/advanced"
    
    threads = []
    
    while has_more:
        params = {
            "page": page,
            "pagesize": 100,
            "order": "desc",
            "sort": "votes",
            "q": question,
            "tagged": ";".join(tags),
            "site": "stackoverflow",
            "key": API_KEY,
            "filter": "!0WRXThY5-I_TB(WxIdhBDpR5e"
        }
        
        try:
            response = requests.get(base_url, params=params)
            print(response.url)
            response.raise_for_status()
            data = response.json()
        
            for item in data.get("items", []):
                if item["answer_count"] >= min_answer_count:
                    thread = {
                        "title": item["title"],
                        "question_body": item["body"],
                        "link": item["link"],
                        "score": item["score"],
                        "tags": item["tags"],
                        "question_id": item["question_id"],
                        "answer_count": item["answer_count"],
                        "view_count": item["view_count"],
                        "comment_count": item["comment_count"],
                        "comments": [],
                        "answers": []
                    }

                    if item["comment_count"] > 0:
                        for comment in item["comments"]:
                            thread["comments"].append({"body":comment["body"]})

                    if item["is_answered"]:
                        answer_url = f'https://api.stackexchange.com/2.3/questions/{item["question_id"]}/answers'
                        params = {
                            "key": API_KEY,
                            'order': 'asc',
                            'sort': 'votes',    
                            'site': 'stackoverflow',   
                            'filter': '!)rlHR4RcMJ3iyACK5j)T',  
                        }
                        response = requests.get(answer_url, params=params)
                        print(response.url)
                        response.raise_for_status()
                        answers = response.json()

                        cleaned_answers = ''
                        for answer in answers['items']:
                            if answer['score'] > 0:
                                thread["answers"].append({
                                    "score": answer["score"],
                                    "body": answer["body"]
                                })

                threads.append(thread)

#             has_more = data.get('has_more', False)
            has_more = False
            page += 1

            # Respect rate limits
            if 'backoff' in response:
                time.sleep(response['backoff'] + 1)
    
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return []
        
    return threads

In [80]:
overlap_tags = ['css', 'html']
question = "overlapping fix"
overlap_qa_threads = fetch_stackoverflow_threads(
    tags=overlap_tags,
    question=question,
    min_answer_count=1
)

https://api.stackexchange.com/2.3/search/advanced?page=1&pagesize=100&order=desc&sort=votes&q=overlapping+fix&tagged=css%3Bhtml&site=stackoverflow&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=%210WRXThY5-I_TB%28WxIdhBDpR5e
https://api.stackexchange.com/2.3/questions/710158/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/8965063/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/3906640/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/21638859/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/13798446/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=v

https://api.stackexchange.com/2.3/questions/51728855/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/53128090/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/66634446/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/67940484/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/3071606/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3iyACK5j%29T
https://api.stackexchange.com/2.3/questions/4265606/answers?key=rl_2AK3hrozTLChbcAFfTMnbu2km&order=asc&sort=votes&site=stackoverflow&filter=%21%29rlHR4RcMJ3

In [101]:
overflow_tags = ['css', 'html']
question = "overflow fix"
overflow_qa_threads = fetch_stackoverflow_threads(
    tags=overflow_tags,
    question=question,
    max_questions=50,
    min_votes=1,
    days=730
)

https://api.stackexchange.com/2.3/search/advanced?q=overflow+fix&order=desc&sort=votes&tagged=css%3Bhtml&site=stackoverflow&pagesize=50&fromdate=1683045598&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=withbody


In [102]:
wrapping_tags = ['css', 'html']
question = "element next line"
wrapping_qa_threads = fetch_stackoverflow_threads(
    tags=wrapping_tags,
    question=question,
    max_questions=50,
    min_votes=1,
    days=730
)

https://api.stackexchange.com/2.3/search/advanced?q=element+next+line&order=desc&sort=votes&tagged=css%3Bhtml&site=stackoverflow&pagesize=50&fromdate=1683045604&key=rl_2AK3hrozTLChbcAFfTMnbu2km&filter=withbody


In [90]:
import json

def save_to_json(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [89]:
file_name = "D:\Codes\Try-LLM\stackoverflow_overlap_threads.json"
save_to_json(overlap_qa_threads, file_name)

print(f"Saved {len(formatted_data)} threads to {file_name}")

Saved 100 threads to D:\Codes\Try-LLM\stackoverflow_overlap_threads.json


In [105]:
file_name = "D:\Codes\Try-LLM\stackoverflow_overflow_threads.json"
save_to_json(overflow_qa_threads, file_name)

print(f"Saved {len(formatted_data)} threads to {file_name}")

24
Saved 24 threads to D:\Codes\Try-LLM\stackoverflow_overflow_threads.json


In [106]:
file_name = "D:\Codes\Try-LLM\stackoverflow_wrapping_threads.json"
save_to_json(wrapping_qa_threads, file_name)

print(f"Saved {len(formatted_data)} threads to {file_name}")

16
Saved 16 threads to D:\Codes\Try-LLM\stackoverflow_wrapping_threads.json


In [8]:
cd

C:\Users\ttaos
